# 用RNN做情意分析

In [1]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### 讀入IMDB電影數據庫

In [3]:
from keras.datasets import imdb

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

colab讀入資料時會發生錯誤，需要更新numpy版本。 [Error in Keras Imdb load_data](https://github.com/tensorflow/tensorflow/issues/28102)

`!pip install numpy==1.16.2`

In [5]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [6]:
print('訓練總筆數:', len(x_train))
print('測試總筆數:', len(x_test))

訓練總筆數: 25000
測試總筆數: 25000


### 輸入的資料
每個數字代表一個英文單字，出現頻率越高數字越小。

In [7]:
# x_train[24999]

### 輸出資料的部分
0為負評 

1為正評

In [8]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

### 將輸入放進神經網路中
* 設定輸入文字長度的上限
* 將每段文字設定為一樣長，太短者補0。

In [9]:
from keras.preprocessing import sequence

In [10]:
x_train = sequence.pad_sequences(x_train,maxlen=150)
x_test = sequence.pad_sequences(x_test,maxlen=150)

In [11]:
x_train.shape

(25000, 150)

### 打造RNN (課堂範例 model)
* 先將 10000 維的文字壓到 N 維
* 然後用 K 個 LSTM 神經元做隱藏層
* 最後一個 output, 直接用 sigmoid 送出

In [12]:
N = 3 # 文字要壓到 N 維
K = 4 # LSTM 有 K 個神經元

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [14]:
model = Sequential()
model.add(Embedding(10000, N))

# K個LSTM的cell
model.add(LSTM(K))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 3)           30000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 128       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 30,133
Trainable params: 30,133
Non-trainable params: 0
_________________________________________________________________


每個cell有3個input 加上 4個前一次的h，等同7個input。

4個神經元有4個bias

因此一個LSTM有 4*7 + 4個

共有K個LSTM => (4*7 + 4) * K

### 訓練

In [17]:
model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
25000/25000 [==============================] - 79s 3ms/step - loss: 0.5050 - acc: 0.7559
Epoch 2/5
25000/25000 [==============================] - 71s 3ms/step - loss: 0.2974 - acc: 0.8871
Epoch 3/5
25000/25000 [==============================] - 75s 3ms/step - loss: 0.2337 - acc: 0.9160
Epoch 4/5
25000/25000 [==============================] - 73s 3ms/step - loss: 0.1970 - acc: 0.9328
Epoch 5/5
25000/25000 [==============================] - 76s 3ms/step - loss: 0.1697 - acc: 0.9451


In [18]:
score = model.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score[0]}')
print(f'測試資料正確率 = {score[1]}')

25000/25000 [==============================] - 10s 401us/step
測試資料的 loss = 0.39434168313503265
測試資料正確率 = 0.84968


### 修改文字維度以及LSTM的神經元個數

### model1

In [19]:
N1 = 100
K1 = 5

model_1 = Sequential()
model_1.add(Embedding(10000,N1))
model_1.add(LSTM(K1))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 2120      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 1,002,126
Trainable params: 1,002,126
Non-trainable params: 0
_________________________________________________________________


In [20]:
(4 * (N1 + K1) + 4) * K1

2120

In [21]:
model_1.fit(x_train, y_train ,  batch_size=32 , epochs=5)

Epoch 1/5
25000/25000 [==============================] - 103s 4ms/step - loss: 0.5095 - acc: 0.7672
Epoch 2/5
25000/25000 [==============================] - 99s 4ms/step - loss: 0.3817 - acc: 0.8448
Epoch 3/5
25000/25000 [==============================] - 103s 4ms/step - loss: 0.3030 - acc: 0.8868
Epoch 4/5
25000/25000 [==============================] - 102s 4ms/step - loss: 0.2689 - acc: 0.8994
Epoch 5/5
25000/25000 [==============================] - 98s 4ms/step - loss: 0.2268 - acc: 0.9181


In [22]:
score_1 = model_1.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score_1[0]}')
print(f'測試資料正確率 = {score_1[1]}')

25000/25000 [==============================] - 19s 755us/step
測試資料的 loss = 0.4179262115955353
測試資料正確率 = 0.83536


### model2

In [23]:
N2 = 100
K2 = 10

model_2 = Sequential()
model_2.add(Embedding(10000,N2))
model_2.add(LSTM(K2))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                4440      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 1,004,451
Trainable params: 1,004,451
Non-trainable params: 0
_________________________________________________________________


In [24]:
(4 * (N2 + K2) + 4) * K2

4440

In [25]:
model_2.fit(x_train, y_train ,  batch_size=32 , epochs=5)

Epoch 1/5
25000/25000 [==============================] - 101s 4ms/step - loss: 0.4447 - acc: 0.7979
Epoch 2/5
25000/25000 [==============================] - 100s 4ms/step - loss: 0.2640 - acc: 0.8955
Epoch 3/5
25000/25000 [==============================] - 99s 4ms/step - loss: 0.1919 - acc: 0.9303
Epoch 4/5
25000/25000 [==============================] - 100s 4ms/step - loss: 0.1471 - acc: 0.9483
Epoch 5/5
25000/25000 [==============================] - 99s 4ms/step - loss: 0.1200 - acc: 0.9580


In [26]:
score_2 = model_2.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score_2[0]}')
print(f'測試資料正確率 = {score_2[1]}')

25000/25000 [==============================] - 19s 762us/step
測試資料的 loss = 0.40902876745700834
測試資料正確率 = 0.85772


### mode3

In [27]:
N3 = 500
K3 = 50

model_3 = Sequential()
model_3.add(Embedding(10000,N3))
model_3.add(LSTM(K3))
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 500)         5000000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                110200    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 5,110,251
Trainable params: 5,110,251
Non-trainable params: 0
_________________________________________________________________


In [28]:
(4 * (N3 + K3) + 4) * K3

110200

In [29]:
model_3.fit(x_train, y_train ,  batch_size=32 , epochs=5)

Epoch 1/5
25000/25000 [==============================] - 289s 12ms/step - loss: 0.4213 - acc: 0.8104
Epoch 2/5
25000/25000 [==============================] - 284s 11ms/step - loss: 0.2586 - acc: 0.8967
Epoch 3/5
25000/25000 [==============================] - 285s 11ms/step - loss: 0.1717 - acc: 0.9340
Epoch 4/5
25000/25000 [==============================] - 286s 11ms/step - loss: 0.1202 - acc: 0.9568
Epoch 5/5
25000/25000 [==============================] - 10850s 434ms/step - loss: 0.1062 - acc: 0.9631


In [30]:
score_3 = model_3.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score_3[0]}')
print(f'測試資料正確率 = {score_3[1]}')

25000/25000 [==============================] - 67s 3ms/step
測試資料的 loss = 0.43849627709388733
測試資料正確率 = 0.85872


### model4

In [31]:
N4 = 500
K4 = 100

model_4 = Sequential()
model_4.add(Embedding(10000,N4))
model_4.add(LSTM(K4))
model_4.add(Dense(1, activation='sigmoid'))
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 500)         5000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               240400    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 5,240,501
Trainable params: 5,240,501
Non-trainable params: 0
_________________________________________________________________


In [32]:
(4 * (N4 + K4) + 4) * K4

240400

In [34]:
model_4.fit(x_train, y_train , batch_size=32 , epochs=5)

Epoch 1/5
25000/25000 [==============================] - 453s 18ms/step - loss: 0.2659 - acc: 0.8930
Epoch 2/5
25000/25000 [==============================] - 470s 19ms/step - loss: 0.1846 - acc: 0.9296
Epoch 3/5
25000/25000 [==============================] - 441s 18ms/step - loss: 0.1217 - acc: 0.9551
Epoch 4/5
25000/25000 [==============================] - 382s 15ms/step - loss: 0.0884 - acc: 0.9688
Epoch 5/5
25000/25000 [==============================] - 380s 15ms/step - loss: 0.0671 - acc: 0.9767


In [35]:
score_4 = model_4.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score_4[0]}')
print(f'測試資料正確率 = {score_4[1]}')

25000/25000 [==============================] - 83s 3ms/step
測試資料的 loss = 0.5386793414115906
測試資料正確率 = 0.84636


### 四組模型對於訓練資料的精準度達到96\~97%左右，但對於測試資料僅於84\~85%的正確率，模型過度擬合訓練資料。  
### 嘗試使用dropout移除神經網路中的部分神經元，防止過度擬合的狀況發生。

### model5

In [36]:
from keras.layers import Dropout

In [37]:
N5 = 500
K5 = 100

model_5 = Sequential()
model_5.add(Embedding(10000,N5))
model_5.add(LSTM(K5))
model_5.add(Dropout(0.5))
model_5.add(Dense(1, activation='sigmoid'))
model_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 500)         5000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               240400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 5,240,501
Trainable params: 5,240,501
Non-trainable params: 0
_________________________________________________________________


In [38]:
(4 * (N5 + K5) + 4) * K5

240400

In [39]:
model_5.fit(x_train, y_train , batch_size=32 , epochs=5)

Epoch 1/5
25000/25000 [==============================] - 405s 16ms/step - loss: 0.3997 - acc: 0.8182
Epoch 2/5
25000/25000 [==============================] - 496s 20ms/step - loss: 0.2800 - acc: 0.8861
Epoch 3/5
25000/25000 [==============================] - 392s 16ms/step - loss: 0.1823 - acc: 0.9321
Epoch 4/5
25000/25000 [==============================] - 399s 16ms/step - loss: 0.1187 - acc: 0.9577
Epoch 5/5
25000/25000 [==============================] - 453s 18ms/step - loss: 0.0923 - acc: 0.9672


In [40]:
score_5 = model_5.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score_5[0]}')
print(f'測試資料正確率 = {score_5[1]}')

25000/25000 [==============================] - 104s 4ms/step
測試資料的 loss = 0.4481125453925133
測試資料正確率 = 0.85436
